In [5]:
### It may be different depending on the environment.

!pip install mlxtend --upgrade --no-deps
!pip install SmilesPE
!pip install rdkit-pypi

Requirement already up-to-date: mlxtend in /usr/local/lib/python3.7/dist-packages (0.18.0)


In [6]:
### It may be different depending on the environment.
### Colab drive mount

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
### It may be different depending on the environment.

import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder

import pickle
from rdkit import Chem

from SmilesPE.pretokenizer import atomwise_tokenizer
from SmilesPE.pretokenizer import kmer_tokenizer

In [10]:
### Example data & tutorial

dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]


In [11]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


In [12]:
apriori(df, min_support=0.2,use_colnames=True)

,support,itemsets
0,0.2,(Apple)
1,0.4,(Corn)
2,0.2,(Dill)
3,0.8,(Eggs)
4,0.2,(Ice cream)
...,...,...
144,0.4,"(Nutmeg, Kidney Beans, Onion, Eggs, Yogurt)"
145,0.2,"(Milk, Nutmeg, Onion, Eggs, Yogurt)"
146,0.2,"(Milk, Nutmeg, Kidney Beans, Onion, Yogurt)"
147,0.2,"(Kidney Beans, Dill, Onion, Eggs, Yogurt, Nutmeg)"


In [13]:
fpgrowth(df, min_support=0.2,use_colnames=True)

,support,itemsets
0,1.0,(Kidney Beans)
1,0.8,(Eggs)
2,0.6,(Yogurt)
3,0.6,(Onion)
4,0.6,(Milk)
...,...,...
144,0.2,"(Ice cream, Onion, Eggs, Corn)"
145,0.2,"(Kidney Beans, Onion, Ice cream, Corn)"
146,0.2,"(Kidney Beans, Corn, Ice cream, Eggs)"
147,0.2,"(Kidney Beans, Onion, Ice cream, Eggs)"


In [14]:
### DILI data(atfer preprocessing, the number of dataset 1036 -> 835)

with open("/content/drive/MyDrive/DILIdata/DILI.pickle","rb") as fr:
    data = pickle.load(fr)
data

,LTKBID,Compound Name,Severity Class,Label Section,vDILIConcern,Version,smiles
0,LT00003,mercaptopurine,8,Warnings and precautions,vMost-DILI-Concern,1,S=C1N=CNc2nc[nH]c12
1,LT00004,acetaminophen,5,Warnings and precautions,vMost-DILI-Concern,2,CC(=O)Nc1ccc(O)cc1
2,LT00006,azathioprine,5,Warnings and precautions,vMost-DILI-Concern,1,Cn1cnc(c1Sc2ncnc3nc[nH]c23)[N+]([O-])=O
3,LT00009,chlorpheniramine,0,No match,vNo-DILI-Concern,2,CN(C)CCC(c1ccc(Cl)cc1)c2ccccn2.OC(=O)\C=C/C(O)=O
4,LT00011,clofibrate,3,Warnings and precautions,vLess-DILI-Concern,1,CCOC(=O)C(C)(C)Oc1ccc(Cl)cc1
...,...,...,...,...,...,...,...
1020,LT02346,clindamycin-2-phosphate,3,Warnings and precautions,Ambiguous DILI-concern,2,CCCC1CC(N(C)C1)C(=O)NC(C(C)Cl)C2OC(SC)C(O[P](O...
1021,LT02356,tyramine,0,No match,vNo-DILI-Concern,2,NCCc1ccc(O)cc1
1031,LT02449,sitaxsentan,8,Withdrawn,vMost-DILI-Concern,2,Cc1cc2OCOc2cc1CC(=O)c3sccc3[S](=O)(=O)Nc4onc(C...
1033,LT02909,dexmethylphenidate,7,Adverse reactions,vLess-DILI-Concern,2,COC(=O)[C@@H]([C@H]1CCCCN1)c2ccccc2


In [15]:
positive=list(data[data["vDILIConcern"]=="vMost-DILI-Concern"]["smiles"])
negative=list(data[data["vDILIConcern"]=="vNo-DILI-Concern"]["smiles"])

In [37]:
n_gram=3
n_augmentation=20

In [38]:
### Positive data handling
positivedb=[]
for i in range(len(positive)):
    if "." not in positive[i]: ### "." make our work complicated. So we except some data including "."
        for j in range(n_augmentation):
            mol = Chem.MolFromSmiles(positive[i])
            newstr = Chem.rdmolfiles.MolToSmiles(mol=mol, doRandom=True)
            positivedb.append(kmer_tokenizer(newstr, ngram=n_gram))

In [39]:
### Negative data handling
negativedb=[]
for i in range(len(negative)):
    if "." not in negative[i]: ### "." make our work complicated. So we except some data including "."
        for j in range(n_augmentation):
            try: ### Some invalied string will make some error
                mol = Chem.MolFromSmiles(negative[i])

                newstr = Chem.rdmolfiles.MolToSmiles(mol=mol, doRandom=True)
                negativedb.append(kmer_tokenizer(newstr, ngram=n_gram))
            except:
                pass

In [40]:
### Frequent pattern mining

In [41]:
min_support=0.25

In [42]:
dataset=positivedb

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [43]:
positive_result = apriori(df, min_support=min_support, use_colnames=True)
positive_result

,support,itemsets
0,0.500000,((=O)
1,0.272293,((C()
2,0.410510,((C))
3,0.251911,((CC)
4,0.253503,((O))
...,...,...
193,0.261146,"(=O), ccc, cc1, cc()"
194,0.277707,"(c(c, c1c, ccc, cc()"
195,0.276433,"(c1c, ccc, cc1, cc()"
196,0.250318,"(ccc, cc(, c1c, =O), 1cc)"


In [44]:
positive_result = fpgrowth(df, min_support=min_support, use_colnames=True)
positive_result

,support,itemsets
0,0.439172,(c2c)
1,0.522611,(c(c)
2,0.609554,(c1c)
3,0.419427,(c1))
4,0.691720,(=O))
...,...,...
193,0.255414,"(c1c, )=O, =O))"
194,0.268471,"((C), C)C)"
195,0.302866,"(c2c, cc2)"
196,0.294586,"(ccc, cc2)"


In [45]:
dataset=negativedb

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [46]:
negative_result = apriori(df, min_support=min_support, use_colnames=True)
negative_result

,support,itemsets
0,0.384677,((=O)
1,0.320430,((C()
2,0.471237,((C))
3,0.431720,((CC)
4,0.364516,((O))
...,...,...
76,0.288172,"(c1), c1c, ccc)"
77,0.260753,"(c1), ccc, cc1)"
78,0.272849,"(c1c, ccc, cc()"
79,0.360484,"(c1c, ccc, cc1)"


In [47]:
negative_result = fpgrowth(df, min_support=min_support, use_colnames=True)
negative_result

,support,itemsets
0,0.542204,(c1c)
1,0.431720,((CC)
2,0.425269,(cc()
3,0.372849,(c1))
4,0.370161,(c(c)
...,...,...
76,0.343817,"((C), C)C)"
77,0.311828,"(ccc, C)C)"
78,0.268817,"(c1c, C)C)"
79,0.281989,"(=O), C)C)"


In [48]:
### Dividing shared patterns(frequent patterns in both of P/N) after mining

In [49]:
positive_result = [ (list(positive_result['itemsets'][i]), positive_result['support'][i]) for i in range(len(positive_result)) ]
negative_result = [ (list(negative_result['itemsets'][i]), negative_result['support'][i]) for i in range(len(negative_result)) ]

In [50]:
share_positive_result = []
pure_positive_result = []
for i in positive_result:
    ch=0
    for j in negative_result:
        if i[0]==j[0]:
            share_positive_result.append(i)
            ch=1
            break
    if ch==0:
        pure_positive_result.append(i)

In [51]:
share_negative_result = []
pure_negative_result = []
for i in negative_result:
    ch=0
    for j in positive_result:
        if i[0]==j[0]:
            share_negative_result.append(i)
            ch=1
            break
    if ch==0:
        pure_negative_result.append(i)

In [52]:
print(len(pure_positive_result))
pure_positive_result

139


[(['O)c'], 0.2984076433121019),
 (['cc2'], 0.3305732484076433),
 (['2cc'], 0.25127388535031847),
 (['c2c', 'c1c'], 0.2843949044585987),
 (['c2c', '=O)'], 0.30668789808917196),
 (['c(c', 'c2c'], 0.28152866242038216),
 (['c2c', 'ccc'], 0.3681528662420382),
 (['c2c', 'cc('], 0.29904458598726114),
 (['c2c', '=O)', 'ccc'], 0.2605095541401274),
 (['c2c', 'ccc', 'cc('], 0.2700636942675159),
 (['c(c', '=O)'], 0.38757961783439493),
 (['c(c', 'ccc', 'cc('], 0.35509554140127386),
 (['c(c', '=O)', 'cc('], 0.31878980891719744),
 (['c(c', '=O)', 'ccc'], 0.3076433121019108),
 (['c(c', '=O)', 'ccc', 'cc('], 0.2735668789808917),
 (['c(c', 'c1c', 'cc('], 0.32579617834394903),
 (['c(c', 'c1c', 'ccc'], 0.3200636942675159),
 (['c(c', '=O)', 'c1c'], 0.2875796178343949),
 (['c(c', 'c1c', 'ccc', 'cc('], 0.27770700636942675),
 (['=O)', 'c1c'], 0.43630573248407645),
 (['=O)', 'c1c', 'ccc'], 0.3703821656050955),
 (['c(c', 'c1)'], 0.3076433121019108),
 (['c1)', 'cc('], 0.3070063694267516),
 (['c1)', '=O)'], 0.305

In [53]:
print(len(pure_negative_result))
pure_negative_result

22


[(['CCN'], 0.29811827956989245),
 (['1)C'], 0.27338709677419354),
 (['CC('], 0.32016129032258067),
 (['CCC'], 0.35940860215053766),
 (['ccc', '(CC'], 0.29596774193548386),
 (['=O)', '(CC'], 0.2986559139784946),
 (['c1c', 'c1)', '1cc'], 0.26801075268817204),
 (['ccc', ')CC'], 0.27123655913978495),
 (['=O)', ')CC'], 0.278494623655914),
 (['C(C', 'ccc'], 0.27419354838709675),
 (['=O)', 'C(C'], 0.27741935483870966),
 (['C(C', '(C)'], 0.26021505376344084),
 (['c1c', '1cc', 'cc1'], 0.3204301075268817),
 (['c1c', '1cc', 'ccc', 'cc1'], 0.2991935483870968),
 (['c1c', '1cc', 'ccc'], 0.3672043010752688),
 (['c1c', '=O)'], 0.28951612903225804),
 ([')=O', '=O)'], 0.3293010752688172),
 (['ccc', 'N(C'], 0.26263440860215054),
 (['ccc', 'C)C'], 0.3118279569892473),
 (['c1c', 'C)C'], 0.26881720430107525),
 (['=O)', 'C)C'], 0.28198924731182795),
 ([')C)', 'C)C'], 0.2653225806451613)]

In [54]:
print(len(share_positive_result))
share_positive_result

59


[(['c2c'], 0.439171974522293),
 (['c(c'], 0.5226114649681529),
 (['c1c'], 0.6095541401273885),
 (['c1)'], 0.4194267515923567),
 (['=O)'], 0.6917197452229299),
 (['ccc'], 0.6910828025477707),
 (['cc('], 0.5837579617834395),
 (['cc1'], 0.5089171974522293),
 (['(=O'], 0.5),
 (['(cc'], 0.3945859872611465),
 (['C(='], 0.39363057324840767),
 ([')cc'], 0.38980891719745225),
 (['O)C'], 0.3178343949044586),
 (['(O)'], 0.2535031847133758),
 (['1cc'], 0.4621019108280255),
 (['(C)'], 0.4105095541401274),
 (['C(C'], 0.32834394904458597),
 ([')=O'], 0.478343949044586),
 ([')C('], 0.2576433121019108),
 ([')C)'], 0.2694267515923567),
 (['C)C'], 0.34808917197452227),
 ([')CC'], 0.2531847133757962),
 (['(C('], 0.27229299363057324),
 (['(CC'], 0.2519108280254777),
 (['N(C'], 0.2531847133757962),
 (['c(c', 'cc('], 0.4194267515923567),
 (['c(c', 'ccc'], 0.41624203821656053),
 (['c(c', 'c1c'], 0.3869426751592357),
 (['c1c', 'ccc'], 0.5101910828025478),
 (['c1)', 'ccc'], 0.343312101910828),
 (['c1)', 'cc1'],

In [55]:
print(len(share_negative_result))
share_negative_result

59


[(['c1c'], 0.5422043010752688),
 (['(CC'], 0.43172043010752686),
 (['cc('], 0.4252688172043011),
 (['c1)'], 0.3728494623655914),
 (['c(c'], 0.37016129032258066),
 (['(O)'], 0.36451612903225805),
 (['(cc'], 0.2760752688172043),
 (['ccc'], 0.6244623655913979),
 ([')CC'], 0.3879032258064516),
 (['cc1'], 0.45026881720430106),
 ([')cc'], 0.29354838709677417),
 (['C(C'], 0.435752688172043),
 (['1cc'], 0.4134408602150538),
 (['O)C'], 0.4344086021505376),
 (['=O)'], 0.5405913978494624),
 (['(=O'], 0.38467741935483873),
 ([')=O'], 0.3760752688172043),
 (['C(='], 0.34381720430107526),
 (['(C('], 0.3204301075268817),
 ([')C('], 0.3053763440860215),
 (['N(C'], 0.3131720430107527),
 (['(C)'], 0.47123655913978496),
 (['C)C'], 0.46344086021505376),
 ([')C)'], 0.3260752688172043),
 (['c2c'], 0.30510752688172044),
 (['c1c', 'ccc'], 0.46586021505376346),
 (['c1c', 'cc('], 0.317741935483871),
 (['ccc', 'cc('], 0.3553763440860215),
 (['c1c', 'ccc', 'cc('], 0.2728494623655914),
 (['c1)', 'c1c'], 0.32473118